In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from collections import Counter
import random
# import matplotlib.pyplot as plt
import numpy as np
# from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm

In [2]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Fuente de la cual queremos obtener los datos
source = 'corenlp'

In [3]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [4]:
ix_to_cat = ['C1', 'C2', 'E1', 'E2', 'E3', 'E4','V1']
cat_to_ix = { cat: ix for ix, cat in enumerate(ix_to_cat) }

num_cat = len(ix_to_cat) # Tamaño de salida de la red

cat_to_ix

{'C1': 0, 'C2': 1, 'E1': 2, 'E2': 3, 'E3': 4, 'E4': 5, 'V1': 6}

In [5]:
ix_to_w = []
with open(f'./../Embeddings/word_labels_{source}.txt', 'r', encoding='utf-8') as f:
    ix_to_w = f.read().splitlines()
    
w_to_ix = { w: ix for ix, w in enumerate(ix_to_w) }

tam_vocab = len(ix_to_w)

ix_to_w[:5]

[',', 'de', 'que', 'la', 'a']

In [6]:
sentences = []
with open(f'./Corpus/Test/test-sample-400.txt', 'r', encoding='utf-8') as f:
    sentences = [s.split() for s in f.read().splitlines()]

[' '.join(s) for s in sentences[:5]]

['Recordó que cuando fue jefe de Gobierno , llevó a cabo un programa para construir 150 mil viviendas en la Ciudad de México y ahora hará un programa de vivienda para ayudar a los afectados por el sismo en este municipio .',
 'Describió que el de sus adversarios tienen que ver con las élites del poder económico y del poder político y el proyecto de millones de mexicanos es la transformación del país , por eso se logra la unidad de todos los partidos , no es solo MORENA , el PT , el PES , sino militantes de todos los partidos , porque solo el pueblo unido y organizado puede salvar a la nación .',
 'Por la tarde , López Obrador estará Tecamachalco y Atlixco , Puebla .',
 '" Qué hicieron estos tecnócratas , acabaron la industria petrolera , va a así en picada la industria petrolera , todavía en el 2004 la producción de petróleo era de 3 millones 400 mil barriles diarios , ahora 2 millones 100 mil barriles diarios , más de un millón de barriles diarios han dejado de producir tan solo en el

In [7]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [8]:
def unk_w_to_ix(w):
    try:
        return w_to_ix[w]
    except KeyError:
        return ixUNK

In [9]:
X = [
    # Convierto las palabras (en minúsculas) a índices
    [ unk_w_to_ix(w.lower()) for w in sent ] 
    for sent in sentences
]

[str(x) for x in X[:5]]

['[169, 2, 124, 144, 761, 1, 31, 0, 2415, 4, 233, 19, 201, 13, 364, 2169, 59, 3378, 8, 3, 111, 1, 20, 7, 120, 312, 19, 201, 1, 901, 13, 1079, 4, 9, 4754, 16, 5, 2150, 8, 65, 533, 6]',
 '[2337, 2, 5, 1, 42, 338, 117, 2, 238, 14, 17, 7473, 15, 109, 386, 7, 15, 109, 699, 7, 5, 256, 1, 95, 1, 81, 18, 3, 160, 15, 32, 0, 16, 60, 11, 4546, 3, 336, 1, 33, 9, 252, 0, 12, 18, 142, 194, 0, 5, 1957, 0, 5, 1991, 0, 226, 718, 1, 33, 9, 252, 0, 25, 142, 5, 58, 1120, 7, 745, 113, 1540, 4, 3, 346, 6]',
 '[16, 3, 231, 0, 29, 30, 223, 7208, 7, 5137, 0, 403, 6]',
 '[10, 107, 692, 301, 3442, 0, 4115, 3, 755, 1062, 0, 43, 4, 99, 8, 2693, 3, 755, 1062, 0, 565, 8, 5, 7473, 3, 1431, 1, 1092, 633, 1, 410, 95, 1128, 59, 2152, 1056, 0, 120, 492, 95, 432, 59, 2152, 1056, 0, 27, 1, 19, 978, 1, 2152, 1056, 106, 3525, 1, 1525, 695, 142, 8, 5, 31, 1, 496, 0, 1, 670, 7, 1, 132, 270, 10, 0, 35, 6]',
 '[169, 2, 46, 24, 5360, 2, 4290, 2, 2314, 1160, 3093, 67, 2250, 4, 42, 7473, 6]']

In [10]:
X = [np.asarray(x, dtype=np.long) for x in X]

In [11]:
def np_to_long_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [12]:
X_test = np_to_long_tensor(X)
N_test = len(X_test)
N_test

49

In [13]:
class PersuasionLSTM(nn.Module):
    def __init__(self, D_in, D_emb, D_lstm, D_out, bidirectional=False):
        super(PersuasionLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        # self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(
            input_size=D_emb, 
            hidden_size=D_lstm,
            bidirectional=bidirectional
        )
        
        self.linear = nn.Linear(
            in_features=D_lstm*2 if bidirectional else D_lstm, 
            out_features=D_out
        )
        
        self.sig = nn.Sigmoid()
        

    def forward(self, sentence):
        T = len(sentence)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        # embeddings = self.dropout(embeddings)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        preact_out = self.linear(lstm_out).view(T, -1)
        return self.sig(preact_out[-1])

In [14]:
# Dimensiones de la red neuronal
D_in = tam_vocab
D_emb = 64
D_lstm = 64
D_out = num_cat

In [15]:
persuasion = PersuasionLSTM(D_in, D_emb, D_lstm, D_out, True).to(device)
persuasion.eval()

PersuasionLSTM(
  (embedding): Embedding(7474, 64)
  (lstm): LSTM(64, 64, bidirectional=True)
  (linear): Linear(in_features=128, out_features=7, bias=True)
  (sig): Sigmoid()
)

In [16]:
dir_path = './modelsaves/'
filename = f'model-V3-seed_69420-last-epoch_100'
persuasion.load_state_dict(torch.load(dir_path+filename))

<All keys matched successfully>

In [17]:
def print_ix_sentence(sentence):
    print(' '.join(ix_to_w[ix] for ix in sentence.data))

def print_ix_cat(probs):
    categorias = []
    print('Categorías inferidas: ', end='')
    for i, p_i in enumerate(probs.data):
        if p_i.item() > 0.51:
            categorias.append(ix_to_cat[i])
    if len(categorias) == 0:
        print('NA')
    else:
        print(', '.join(categorias))
    print('\nProbabilidades')
    print('\t'.join(ix_to_cat))
    print('\t'.join([f'{p_i.item():0.3f}' for p_i in probs.data]))

1. **Construcción del emisor/candidato (C1):** el candidato se coloca como referente principal del acto discursivo para exaltar sus cualidades personales y erigirse, a los ojos de sus interlocutores, como un líder en quien es adecuado confiar para el desarrollo de acciones de gobierno a favor de la sociedad; esto lo lleva a cabo mediante realizaciones léxicas de la primera persona, en singular o plural, y formas flexivas de verbos y posesivos para referirse a sí mismo, ya sea como individuo o como miembro de una colectividad. Para el caso de la campaña presidencial que nos ocupa, las cualidades del candidato emitidas por un tercero en el marco de una tarea proselitista, quedarán incluidas en esta categoría, ya que contribuyen a construir la imagen positiva del contendiente.

2. **Promesa de campaña (C2):** a través de piezas discursivas, orales o escritas, el candidato oferta, destaca o compromete acciones que proyecta convertir en políticas públicas a favor de la población, si obtiene el triunfo en las urnas, con lo que implícitamente solicita el voto ciudadano que lo llevaría al poder.

3. **Construcción del adversario (E1):** el mensaje es dirigido a su contraparte en la contienda electoral y está construido a manera de réplica y puede usar diferentes figuras retóricas (metáfora, ironía, metonimia) para conformar en el receptor una imagen determinada del adversario, que esencialmente tiene finalidad de descrédito.

4. **Exageración de la información (E2):** se destacan los datos favorables a los fines persuasivos, con el objetivo de crear una idea positiva en la mente del receptor referente al tema del que se habla, o incluso negativa respecto de información relativa al adversario; el persuasor desfigura el sentido original del acontecimiento, nos dice Roiz (1994), mediante códigos diferentes: humorístico, burlesco, cínico, entre otros.

5. **Recurso retórico (E3):** son variables basadas en figuras retóricas (metáfora, ironía, hipérbole, personificación, pleonasmo, perífrasis) que pueden fortalecer el efecto persuasivo, de acuerdo con Reardon (1991).

6. **Apelación al miedo (E4):** el candidato emite mensajes dentro de su alocución que pretenden provocar sentimientos de aprensión, desasosiego o preocupación con respecto al adversario o a sus propuestas. 

7. **Llamado al voto (V1):** el emisor del mensaje hace una invitación directa al ciudadano para que apoye su proyecto político el día de la elección al depositar el sufragio a su favor.


In [18]:
pred.view(-1)

NameError: name 'pred' is not defined

In [19]:
with torch.no_grad():
    for i, x in enumerate(X_test):
        print('='*50)
        print(' '.join(sentences[i]))
#         print_ix_sentence(x)
        pred = persuasion(x)
        print()
        print_ix_cat(pred)
        print('='*50)
        print()

Recordó que cuando fue jefe de Gobierno , llevó a cabo un programa para construir 150 mil viviendas en la Ciudad de México y ahora hará un programa de vivienda para ayudar a los afectados por el sismo en este municipio .

Categorías inferidas: C2

Probabilidades
C1	C2	E1	E2	E3	E4	V1
0.003	0.999	0.000	0.004	0.000	0.000	0.000

Describió que el de sus adversarios tienen que ver con las élites del poder económico y del poder político y el proyecto de millones de mexicanos es la transformación del país , por eso se logra la unidad de todos los partidos , no es solo MORENA , el PT , el PES , sino militantes de todos los partidos , porque solo el pueblo unido y organizado puede salvar a la nación .

Categorías inferidas: C1

Probabilidades
C1	C2	E1	E2	E3	E4	V1
0.943	0.000	0.000	0.000	0.000	0.000	0.000

Por la tarde , López Obrador estará Tecamachalco y Atlixco , Puebla .

Categorías inferidas: NA

Probabilidades
C1	C2	E1	E2	E3	E4	V1
0.004	0.000	0.001	0.000	0.000	0.000	0.025

" Qué hicieron es

0.000	1.000	0.000	0.000	0.000	0.000	0.001

Manifestó que no se tiene que gastar tanto en la construcción del aeropuerto en el Lago de Texcoco , porque hasta ahora es una inversión estimada de 300 mil millones de pesos , pero se puede consumir mucho más dinero del presupuesto , de todos los ciudadanos .

Categorías inferidas: E2

Probabilidades
C1	C2	E1	E2	E3	E4	V1
0.011	0.149	0.001	0.899	0.415	0.000	0.000

En conferencia de prensa , Ricardo Anaya señaló que esta nueva amenaza del Presidente de Estados Unidos , realizada sin provocación alguna , requiere de una respuesta rápida y enérgica , lo que hace inaceptable la pasividad del gobierno mexicano .

Categorías inferidas: E1

Probabilidades
C1	C2	E1	E2	E3	E4	V1
0.002	0.002	0.862	0.165	0.002	0.000	0.000

En un encuentro con cientos de empresarios y representantes de la sociedad tamaulipeca , José Antonio Meade aseguró que " México debe cambiar , y debemos comenzar por cambiar la estrategia de seguridad , porque ésta ha fracasado . No po